# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

import sqlite3
import pandas as pd
import re


from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from xgboost import XGBClassifier

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /Users/MichaelPa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/MichaelPa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/MichaelPa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/MichaelPa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# load data from database
cxn = sqlite3.connect('../data/dsnd_disaster.db')
df = pd.read_sql('select * from messages_1', cxn)
X = df.iloc[:,1]
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pd.DataFrame(pipeline.predict(X_test))

In [12]:
def print_score(y_test, y_pred):
    #Prints out the average scores of all categories
    scores = []
    for i in range(0,36):
        tmp=precision_recall_fscore_support(y_test.iloc[:,i], y_pred.iloc[:,i], average='binary')
        scores.append(tmp)
    scores = pd.DataFrame(scores)
    scores.columns = ['precision','recall','fscore','support']
    scores.drop(columns = 'support',inplace = True)
    scores['categories'] = y_test.columns
    scores.set_index('categories', inplace = True)
    scores = scores[scores.precision>0] #eliminating empty rows

    print('precision:%.2f, recall:%.2f, fscore:%.2f' % (scores.precision.mean(), scores.recall.mean(), scores.fscore.mean()))
    
    return

print_score(y_test, y_pred)

precision:0.69, recall:0.25, fscore:0.32


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0)
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    #'clf__estimator__n_estimators': [50, 100, 200]
    }

cv = GridSearchCV(pipeline, param_grid=parameters,  cv=5, n_jobs=-1,scoring='f1_weighted')
cv.fit(X_train, y_train)
y_pred = pd.DataFrame(cv.predict(X_test))

print_score(y_test,y_pred)

precision:0.67, recall:0.22, fscore:0.28


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
from sklearn.utils import parallel_backend

with parallel_backend('multiprocessing'):
    print('Building model...')
    pipeline2 = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(XGBClassifier(objective = 'binary:hinge')))
        ])

    parameters2 = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        #'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__max_depth': [5,10,20]}

    cv2 = GridSearchCV(pipeline2, param_grid=parameters2,  cv=5, n_jobs=-1, scoring='f1_weighted')
    
    print('Training model...')
    cv2.fit(X_train, y_train)

    print('Evaluating model...')
    y_pred = pd.DataFrame(cv.predict(X_test))
    print_score(y_test,y_pred)

Building model...
Training model...


### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(cv2, '../models/pipelinemodel.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.